# Import libraries

In [1]:
import numpy as np
from warnings import filterwarnings
from cvxpy import Variable, Problem, Maximize, Minimize, log, quad_form
from pulp import LpProblem, LpVariable, LpInteger, LpMaximize, LpMinimize

filterwarnings("ignore")

# 1. Example of convex-concave optimization using `cvxpy`
Before going to the section of `pulp`, we will focus on `cvxpy` and how to establish an optimization-problem with this library!

Also, we have 3 important features to determine : `input_variable`, `objective function` and `constraints`
### 1.1. Define/assign the `input variables`
In this section, we will only consider the optimization problem $(P)$ on $\mathbb{R}^3$, where
$$ x = \left( x_1, x_2, x_3 \right) $$

In [2]:
x1 = Variable()
x2 = Variable()
x3 = Variable()

### 1.2. Define the `objective function`
Here we use 2 type `convex` and `concave` functions

In [3]:
obj_func_1 = Minimize((x1 + x2 - x3)**2)
obj_func_2 = Maximize(log(x1 + x2 + x3 + 1))

### 1.3. Define the contrains

$$ \left\lbrace \begin{array}{clc} x_1 + x_2 &=& 2 \\ x_2 + x_3 & \geq & - 1 \\ x_1 - 2x_3 &=& 0 \end{array} \right. $$

In [4]:
constraints = [x1 + x2 == 2, x2 + x3 >= -1, x1 - 2*x3 == 0]

In [5]:
print("Convex optimization")
prob = Problem(obj_func_1, constraints)
prob.solve()
print(f"x1_opt = {x1.value} \t x2_opt = {x2.value} \t x3_opt = {x3.value}")

Convex optimization
x1_opt = 4.000000000000001 	 x2_opt = -2.000000000000001 	 x3_opt = 2.0000000000000004


In [6]:
print("Concave optimization")
prob = Problem(obj_func_2, constraints)
prob.solve()
print(f"x1_opt = {x1.value} \t x2_opt = {x2.value} \t x3_opt = {x3.value}")

Concave optimization
x1_opt = 5.999999713449373 	 x2_opt = -3.9999997332886355 	 x3_opt = 2.999999856727714


### 1.4. Wrapping up all together with the vector-form

- For example, if $x = \left( x_1,x_2,x_3 \right)^T$ then the first objective-function becomes

$$ F(x) = (x_1 + x_2 - x_3)^2 = x^T A x $$

where 
$$ A = a^Ta = \left( \begin{array}{ccc} 1 & 1 & -1 \\ 1 & 1 & -1 \\ -1 & -1 & 1 \end{array} \right)$$
and 
$$ a=(1,1,-1) $$

- Likewise, we have 3 distinct contraints here, 
$$ c_1 x = 2 \qquad c_2 x \geq -1 \qquad c_3 x = 0 $$
where
$$ c_1 = (1,1,0) \qquad c_2 = (0,1,1) \qquad c_3 = (1,0,-2) $$

In [7]:
a = np.array([[1,1,-1]])
c1 = np.array([1,1,0])
c2 = np.array([0,1,1])
c3 = np.array([1,0,-2])
x = Variable(3)
vec_cons = [c1@x == 2, c2@x >= -1, c3@x == 0]
Sigma = a.T.dot(a) # equivalent to a.T*a
vec_objf_1 = Minimize(quad_form(x, Sigma)) # equivalent to x.T@(a.T*a)@x
print("Final result")
prob = Problem(vec_objf_1, vec_cons)
prob.solve()
x.value

Final result


array([ 4., -2.,  2.])

- Now, how to do the samething for the other problem, the objective function in this case will become

$$ G(x) = \log(x_1 + x_2 + x_3 + 1) = \log (bx + 1) $$

where $b = (1,1,1)$

In [8]:
b = np.array([1,1,1])
vec_objf_2 = Maximize(log(b@x + 1))
print("Final result")
prob = Problem(vec_objf_2, vec_cons)
prob.solve()
x.value

Final result


array([ 5.99999971, -3.99999973,  2.99999986])

# 2. Using `pulp`

In [9]:
x1 = LpVariable("x1", 0, None, LpInteger)
x2 = LpVariable("x2", 0, None, LpInteger)
x3 = LpVariable("x3", 0, None, LpInteger)

prbl_1 = LpProblem("convex_opt", LpMaximize)

In [10]:
# Create 3 variables 
x1=LpVariable("x1",0,None,LpInteger)
x2=LpVariable("x2",0,None,LpInteger)
x3=LpVariable("x3",0,None,LpInteger)

In [11]:
model = LpProblem("convex_opt", LpMaximize)

# Create maximize objective function
model += (x1 + x2 - x3)*(x1 + x2 - x3)

# Create three constraints
model += x1+x2==2
model += x2 + x3 >= -1
model += x1 - 2*x3 == 0

# The problem is solved using PuLP's choice of Solver
model.solve()

for v in model.variables():
    print(v.name, "=", v.varValue)

TypeError: Non-constant expressions cannot be multiplied

=> So noting that when using `pulp`, we can not solve the non-linear objective function. This only good when we use it in the linear-optimization problem. 

In [14]:
model2 = LpProblem("linear_opt", LpMaximize)

# Create 3 variables tables, chairs, and bookcases 
x1 = LpVariable("tables", 0, None, LpInteger)
x2 = LpVariable("chairs", 0, None, LpInteger)
x3 = LpVariable("bookcases", 0, None, LpInteger)

# Create maximize objective function
model2 += 40 * x1 + 30 * x2 + 45 * x3

# Create three constraints
model2 += 2 * x1 + 1 * x2 + 2.5 * x3 <= 60, "Labour"
model2 += 0.8 * x1 + 0.6 * x2 + 1.0 * x3 <= 16, "Machine"
model2 += 30 * x1 + 20 * x2 + 30 * x3 <= 400, "wood"
model2 += x1 >= 10, "tables"

# The problem is solved using PuLP's choice of Solver
model2.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/codespace/.python/current/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7b387ed37cd34d06ad79bf88914e1c09-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/7b387ed37cd34d06ad79bf88914e1c09-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 29 RHS
At line 34 BOUNDS
At line 38 ENDATA
Problem MODEL has 4 rows, 3 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 550 - 0.00 seconds
Cgl0004I processed model has 2 rows, 3 columns (3 integer (0 of which binary)) and 6 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -550 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -550, took 0 iterations and 0 nodes (0.00 seconds)
Cbc003

1

In [15]:
for v in model2.variables():
    print(v.name, "=", v.varValue)

bookcases = 0.0
chairs = 5.0
tables = 10.0
